# Time based randomness

Create a Python program to decrypt the contents of a given ciphertext encrypted in AES-128-ECB by trying to guess the key, which was generated using the code shown below:

    import random
    import time
    r = random.Random(int(time.time()))
    bits = r.getrandbits(128)
    key = bits.to_bytes(16, 'big')


Your program should also take as command line arguments a minimum Unix time and a maximum Unix time, as a range and try iterating between these two points as seed values for the random function. The dictionary “dictionary_1000.txt” and catching of padding exceptions can be used to determine the successful decryption of the given ciphertext. PKCS7 padding should be used for this exercise.

Your program should support the following arguments:

● -c ciphertext file: Ciphertext file

● -max Unix time: Max Unix time

● -min Unix time: Min Unix time


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import argparse
import random
import sys

In [ ]:

def get_key(seed: int):
    r = random.Random(seed)
    bits = r.getrandbits(128)
    key = bits.to_bytes(16, 'big')

    return key


def decrypt(ciphertext: bytes, key: bytes):
    cipher = Cipher(algorithms.AES128(key), modes.ECB())
    decryptor = cipher.decryptor()

    padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    return unpadder.update(padded_data)+unpadder.finalize()


In [ ]:

def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-c", "--ciphertext", help="Ciphertext (as a hexadecimal string)", type=str, required=True)
    parser.add_argument("--max", help="Max UNIX time", type=int, required=True)
    parser.add_argument("--min", help="Min UNIX time", type=int, required=True)

    args = parser.parse_args(arguments)

    cipher = bytes.fromhex(args.ciphertext)
    print("ciphertext:"+str(cipher))
    max_attempt = {'seed': 0, 'score': 0, 'result': "", 'key': ""}

    with open("dictionary_1000.txt") as dictionary:
        dict_words = dictionary.read().encode().splitlines()

    for i in range(args.min, args.max+1):
        try:
            key = get_key(i)
            attempt = decrypt(cipher, key)
        except ValueError:
            pass
        else:
            count = 0
            for word in attempt.split():
                if word.lower() in dict_words:
                    count += 1

            if count > max_attempt["score"]:
                max_attempt["seed"] = i
                max_attempt["key"] = key
                max_attempt["score"] = count
                max_attempt["result"] = attempt

    if max_attempt["seed"]:
        print(f"Possible recovered key: {max_attempt['key'].hex()}")
        print(f"Possible key generation time: {max_attempt['seed']}")
        print(f"Possible plaintext:\n{max_attempt['result'].decode()}")




In [ ]:

sys.argv = [
    "", 
    "-c", "3bfca22088615b4838e1721d1a23a34bdb95e111583129731af3d228d618cefc5bca25f37cd2dcdc275556dc3cdf93b74a2ff5ec59aac5460d86118c2ec17dc2a4588a8609291f81f64f692ab817302629604dd05d652979daa70aa0a891605fdda44dab97d3f7c05e451af5edc6fbb89ac3b9cabe885e1270b64452bd252c2bf2716d5529997038f5e888ad798626214ea845ed731d2ca62c4776c7deac0564",
    "--max", "1738368000",
    "--min", "1600000000"
]
main(sys.argv[1:])
